# Sensitivity study

Author: Raghav Kansal

In [1]:
import logging
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from boostedhh import utils, hh_vars
from bbtautau.bbtautau_vars import HLT_dict, HLT_list

logger = logging.getLogger("boostedhh.utils")
logger.setLevel(logging.DEBUG)

In [2]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [3]:
MAIN_DIR = Path("../../../")

plot_dir = MAIN_DIR / "plots/SensitivityStudy/24Nov20"
plot_dir.mkdir(parents=True, exist_ok=True)

year = "2022"
data_samples_tag = "24Nov18Data_v12_private_signal"
signal_samples_tag = "24Nov9UpdateTriggers_v12_private_signal"

## Define and load samples

In [4]:
base_dir = Path("/ceph/cms/store/user/rkansal/bbtautau/skimmer/")

samples = {
    "jetmet": utils.Sample(
        path=base_dir / data_samples_tag,
        selector="JetHT|JetMET",
        label="JetMET",
        isData=True,
        year=year,
    ),
    "tau": utils.Sample(
        path=base_dir / data_samples_tag,
        selector="Tau_Run",
        label="Tau",
        isData=True,
        year=year,
    ),
    "bbtt": utils.Sample(
        path=base_dir / signal_samples_tag,
        selector=hh_vars.bbtt_sigs["bbtt"],
        label=r"HHbb$\tau\tau$",
        isData=False,
        year=year,
    ),
}

In [ ]:
pt_cut = 250
# msd_cut = 40

filters = [
    [
        ("('ak8FatJetPt', '0')", ">=", pt_cut),
        ("('ak8FatJetPt', '1')", ">=", pt_cut),
        # ("('ak8FatJetMsd', '0')", ">=", msd_cut),
        # ("('ak8FatJetMsd', '1')", ">=", msd_cut),
        # ("('ak8FatJetPNetXbb', '0')", ">=", 0.8),
    ],
]

# save cutflow as pandas table
cutflow = pd.DataFrame(index=list(samples.keys()))

# dictionary that will contain all information (from all samples)
events_dict = {}
for key, sample in samples.items():
    events_dict[key] = utils.load_sample(sample, filters)

utils.add_to_cutflow(events_dict, "Preselection", "finalWeight", cutflow)
cutflow

## Triggers